![CH5-ADS.png](.\Media\CH5-ADS.png)
# <span style="color:#cc5500;">Simple Repair Techniques</span>

The purpose of Repair, is to make the database structurally consistent and to do to this as fast and safely as possible.  Usually you’ll use repair because you do not have the backups necessary to restore your database with no data loss.  Be careful using repair or manually fixing corruptions as it may involve data loss and I recommend that you practice using repair or manually fixing corruptions before doing it for real in production.  

In this Notebook we’ll cover:

1. How repair works
2. Repair options
3. Manually fixing some corruptions

- REPAIR\_ALLOW\_DATA\_LOSS | REPAIR\_FAST | REPAIR\_REBUILD
    - Specifies that DBCC CHECKDB repair the found errors. Use the REPAIR options only as a last resort. The specified database must be in single-user mode to use one of the following repair options.
- REPAIR\_ALLOW\_DATA\_LOSS
    - Tries to repair all reported errors. These repairs can cause some data loss.
    - The REPAIR\_ALLOW\_DATA\_LOSS option is a supported feature but it may not always be the best option for bringing a database to a physically consistent state. If successful, the REPAIR\_ALLOW\_DATA\_LOSS option may result in some data loss. In fact, it may result in more data lost than if a user were to restore the database from the last known good backup.
- REPAIR\_FAST
    - Maintains syntax for backward compatibility only. No repair actions are performed.
- REPAIR\_REBUILD
    - Performs repairs that have no possibility of data loss. This can include quick repairs, such as repairing missing rows in nonclustered indexes, and more time-consuming repairs, such as rebuilding an index.
    - This argument does not repair errors involving FILESTREAM data.

Microsoft always recommends a user restore from the last known good backup as the primary method to recover from errors reported by DBCC CHECKDB. The REPAIR\_ALLOW\_DATA\_LOSS option is not an alternative for restoring from a known good backup. It is an emergency "last resort" option recommended for use only if restoring from a backup is not possible.

Certain errors, that can only be repaired using the REPAIR\_ALLOW\_DATA\_LOSS option, may involve deallocating a row, page, or series of pages to clear the errors. Any deallocated data is no longer accessible or recoverable for the user, and the exact contents of the deallocated data cannot be determined. Therefore, referential integrity may not be accurate after any rows or pages are deallocated because foreign key constraints are not checked or maintained as part of this repair operation. The user must inspect the referential integrity of their database (using DBCC CHECKCONSTRAINTS) after using the REPAIR\_ALLOW\_DATA\_LOSS option.

Before performing the repair, create physical copies of the files that belong to this database. This includes the primary data file (.mdf), any secondary data files (.ndf), all transaction log files (.ldf), and other containers that form the database including full text catalogs, file stream folders, memory optimized data, etc.

Before performing the repair, consider changing the state of the database to EMERGENCY mode and trying to extract as much information possible from the critical tables and save that data.

## <span style="color:#cc5500;">How does Repair work?</span>

- What is the purpose of repair?
    - The purpose of repair is NOT to try and save all of your data.  Make the database structurally consistent so that the Storage Engine can process the database without running into other corruptions
    - Tries to be as fast as possible which is why most of the repairs in the REPAIR\_ALLOW\_DATA\_LOSS option are 'delete what is broken and fix up all of the links'
- How does it know what to repair?
    - Each stage of running CHECKDB it makes a list of corruptions it has found, and it processes those corruptions.
- How does it process those corruptions and how does it choose what to repair first?
    - It decides that based upon what corruptions there are
    - Every corruption has a ranking of how intrusive of a repair it will be.  It processes the most intrusive errors first, because many times, when the most intrusive error is processed, it results in being able to cross some of the more minor issues off the list
- Did it repair everything?
    - Check the output.  At the bottom of the output it will display the count of errors: found x number of errors and fixed y number of errors.  If the two numbers match, then it thinks it fixed everything
    - Be careful –some corruptions could be masked by others
- Why do we have to put the database in Single User Mode.  Why aren’t repairs done while online?
    - It is hard enough to get them right while offline, that it is simply too difficult if the database were online, to keep up with and track all of the changes that occurs on an online database

## <span style="color:#cc5500;">Be aware of REPAIR_ALLOW_DATA_LOSS</span>

- It was very deliberately named
- It usually fixes structural inconsistencies by de-allocating
    - This is the fastest and most provably correct way
- It doesn’t take into account:
    - Foreign-key constraints
    - Inherent business logic and data relationships
    - Replication
- Before running repair, protect yourself
    - Take a backup and quiescereplication topologies involved
- After running repair, check the data
    - Run DBCC CHECKDB again to make sure all corruptions were repaired
    - Run DBCC CHECKCONSTRAINTS if necessary
    - Reinitialize any replication topologies involved